In [26]:
%pip install langchain
%pip install langchain-openai
%pip install langchain-community
%pip install langchainhub
%pip install pypdf
%pip install faiss-cpu

^C


For this notebook besides the OPENAI API KEY, you will need the LANGCHAIN API KEY (if you want to use langsmith platform for evaluation) and the TAVILY API KEY. 

1. [See here for the quick setup for LANGSMITH](https://docs.smith.langchain.com/).
2. [See here for the quick setup for TAVILY API KEY](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/).

In [ ]:
import os
import getpass

# # Set OPENAI API Key

os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["TAVILY_API_KEY"] = getpass.getpass() # you can obtain your tavily api key here: https://tavily.com/
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass() # obtain your langsmith api key here: https://smith.langchain.com/settings
# OR (load from .env file)
# make sure you have python-dotenv installed
# from dotenv import load_dotenv
# load_dotenv("./.env")

In [28]:
import os
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass() # obtain your langsmith api key here: https://smith.langchain.com/settings


Create your api key in the settings page of the [langsmith platform here](https://smith.langchain.com/settings).

> Summarized from this awesome video about the discussion of chains and agents: https://www.youtube.com/watch?v=bYLHklxEd_k&t=1801s


- Chains and agents represent two concepts in technology with distinct functionalities. Chains refer to linear sequences of events or processes that rely on the step-by-step execution of tasks, often requiring predefined inputs and producing predictable outcomes. 

- Agents, conversely, are autonomous systems capable of making decisions and performing tasks independently, adapting to new situations without explicit instructions. While chains excel in structured environments with clear rules, agents are designed to navigate complex, dynamic scenarios, learning and adjusting their behavior over time. The core difference lies in the level of autonomy and complexity each can handle, with agents typically seen as more advanced due to their self-governing nature.

Making a reporting agent.

Let's build an LLM agent that can build simple reports for tasks performed just by reading through the code and files inside a folder + notes from Notion page or .txt file.

First, let's organize our thoughts, an agent will be composed of:

- Tool
- Base LLM
- Conditions for stoping and returning an output 

![](2023-10-27-19-28-10.png)

In [29]:
MODEL='gpt-4o-mini'
TEMP=0.0

In [30]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [31]:
search = TavilySearchResults()

In [32]:
search.invoke("what is the weather in Paris?")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Paris', 'region': 'Ile-de-France', 'country': 'France', 'lat': 48.87, 'lon': 2.33, 'tz_id': 'Europe/Paris', 'localtime_epoch': 1724702037, 'localtime': '2024-08-26 21:53'}, 'current': {'last_updated_epoch': 1724701500, 'last_updated': '2024-08-26 21:45', 'temp_c': 19.1, 'temp_f': 66.4, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 2.5, 'wind_kph': 4.0, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1022.0, 'pressure_in': 30.18, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 52, 'cloud': 0, 'feelslike_c': 19.1, 'feelslike_f': 66.4, 'windchill_c': 19.6, 'windchill_f': 67.3, 'heatindex_c': 19.6, 'heatindex_f': 67.3, 'dewpoint_c': 9.1, 'dewpoint_f': 48.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 8.0, 'gust_kph': 12.9}}"},
 {'url': 'https://world-weather.info/forecast/france/paris/26-august/',
  'content': 'Det

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction")
docs = loader.load()
docs

In [ ]:
docs[0]

Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentLangChain 0.2 is out! Leave feedback on the v0.2 docs here. You can view the v0.1 docs here.IntegrationsAPI referenceLatestLegacyMorePeopleContributingCookbooks3rd party tutorialsYouTubearXivv0.2v0.2v0.1🦜️🔗LangSmithLangSmith DocsLangChain HubJS/TS Docs💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector st

In [ ]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

documents

[Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Skip to main contentLangChain 0.2 is out! Leave feedback on the v0.2 docs here. You can view the v0.1 docs here.IntegrationsAPI referenceLatestLegacyMorePeopleContributingCookbooks3rd party tutorialsYouTubearXivv0.2v0.2v0.1🦜️🔗LangSmithLangSmith DocsLangChain HubJS/TS Docs💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBui

In [ ]:
vector = FAISS.from_documents(documents, OpenAIEmbeddings())

In [ ]:
retriever = vector.as_retriever()

In [ ]:
retriever.invoke("What is langchain?")[0]

Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain')

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langchain_search",
    "Search for information about LangChain. For any questions about LangChain, you must use this tool!",
)

In [ ]:
tools = [search, retriever_tool]

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=MODEL, temperature=TEMP)

llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-e1de4012-a9d3-4d12-a490-20850fb44fcf-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

In [ ]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant'))

In [ ]:
prompt.messages[1]

MessagesPlaceholder(variable_name='chat_history', optional=True)

In [ ]:
prompt.messages[2]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))

In [ ]:
prompt.messages[3]

MessagesPlaceholder(variable_name='agent_scratchpad')

In [ ]:
tools

[TavilySearchResults(),
 Tool(name='langchain_search', description='Search for information about LangChain. For any questions about LangChain, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x116bc7b00>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x123dd0450>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x116bc7c40>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x123dd0450>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))]

In [ ]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "What are some useful applications you can build with LangChain? List 5 in bullet points."})



> Entering new AgentExecutor chain...

Invoking: `langchain_search` with `{'query': 'useful applications of LangChain'}`


ConversationalRetrievalChainMigrating from LLMChainMigrating from LLMMathChainMigrating from LLMRouterChainMigrating from MapReduceDocumentsChainMigrating from MapRerankDocumentsChainMigrating from MultiPromptChainMigrating from RefineDocumentsChainMigrating from RetrievalQAMigrating from StuffDocumentsChainSecurityIntroductionOn this pageIntroductionLangChain is a framework for developing applications powered by large language models (LLMs).LangChain simplifies every stage of the LLM application lifecycle:Development: Build your applications using LangChain's open-source building blocks, components, and third-party integrations.

Use LangGraph to build stateful agents with first-class streaming and human-in-the-loop support.Productionization: Use LangSmith to inspect, monitor and evaluate your chains, so that you can continuously optimize and deploy with confide

{'input': 'What are some useful applications you can build with LangChain? List 5 in bullet points.',
 'output': 'Here are five useful applications you can build with LangChain:\n\n- **Chatbots**: Create conversational agents that can interact with users, providing information and assistance in a natural language format.\n\n- **Question Answering Systems**: Develop applications that can answer questions based on a specific dataset or knowledge base, such as a graph database or SQL data.\n\n- **Retrieval-Augmented Generation (RAG) Apps**: Build applications that enhance the generation of text by retrieving relevant information from external sources to provide more accurate and contextually relevant responses.\n\n- **PDF Ingestion and Q&A Systems**: Create systems that can process PDF documents, extract information, and allow users to ask questions about the content.\n\n- **Local RAG Applications**: Develop applications that utilize local data sources for retrieval and generation, enabli

Now we can inspect what is going on in the [langsmith platform!](https://smith.langchain.com/o/b68c89e6-dae0-54ab-a84b-3201d39a3c50/settings).